## <CENTER>  Mätning av $Z$-bosonens massa med ATLAS-data

Börja med att hämta datan och ladda in de i ROOT-träd.

In [2]:
import ROOT

Welcome to JupyROOT 6.14/04


In [4]:
# Leta på följande webbadress, där ni bör hitta två filer, en för händelser med elektroner och en för myoner
# http://opendata.atlas.cern/release/samples
data_file_electrons = ROOT.TFile.Open("http://opendata.atlas.cern/release/samples/Data/DataEgamma.root")
data_file_electrons_simulation = ROOT.TFile.Open("http://opendata.atlas.cern/release/samples/MC/mc_147770.Zee.root")
data_file_muons = ROOT.TFile.Open("http://opendata.atlas.cern/release/samples/Data/DataMuons.root")

In [5]:
# Ladda in i träd (TFile::ls() kan användas för att lista filens innehåll)
tree_electrons = data_file_electrons.Get("mini")
tree_electrons_simulation = data_file_electrons_simulation.Get("mini")
tree_muons = data_file_muons.Get("mini")

In [6]:
def getInvMass(lep1_pt, lep1_eta, lep1_phi, lep2_pt, lep2_eta, lep2_phi):
    ''' räkna ut invarianta massan med formeln i Introduktions-notboken (antar masslösa partiklar) '''
    import math
    msq = 2*lep1_pt*lep2_pt*(math.cosh(lep1_eta-lep2_eta) - math.cos(lep1_phi-lep2_phi))
    return math.sqrt(msq)

In [7]:
def getInvMassTL(lep1_pt, lep1_eta, lep1_phi, lep1_e, lep2_pt, lep2_eta, lep2_phi, lep2_e):
    ''' räkna ut invarianta massan med TLorentzVectors '''
    lep1=ROOT.TLorentzVector()
    lep2=ROOT.TLorentzVector()
    lep1.SetPtEtaPhiE(lep1_pt, lep1_eta, lep1_phi, lep1_e)
    lep2.SetPtEtaPhiE(lep2_pt, lep2_eta, lep2_phi, lep2_e)
    invmass=lep1+lep2
    return invmass.M()

In [8]:
# Definiera binnar
nbins=50
lowedge=30.
upedge=150.

In [9]:
# Loopa över alla händelser, räkna ut den invarianta massan av elektron-positron-par och myon-antimyon-par
# Detta kan t.ex. göras med hjälp av ROOT-klassen TLorentzVector eller genom att skriva en egen funktion.
# Fyll histogram med den invarianta massan.
# Använd vad ni lärde dig i första Introduktion-notebook.
# Nu tittar ni på data, så varje händelse kommer inte att innehålla en Z-boson.
# Ni måste själva sortera ut de händelser som har de karakteristiska drag man förväntar sig för Z-bosoner.
h_mass_electrons = ROOT.TH1F("h_mass_electrons", "; Invariant mass [GeV]; Number of events", nbins, lowedge, upedge)
h_mass_muons = ROOT.TH1F("h_mass_muons", "; Invariant mass [GeV]; Number of events", nbins, lowedge, upedge)
ievt = 0
for evt in tree_electrons:
    if evt.lep_n != 2: continue
    if not (evt.lep_type[0]==11 and evt.lep_type[1]==11): continue
    if (evt.lep_charge[0] + evt.lep_charge[1]) != 0: continue 
    if ievt > 4000:
        break
        
    # invmass_func =     getInvMass(evt.lep_pt[0]*1e-3, evt.lep_eta[0], evt.lep_phi[0],                    evt.lep_pt[1]*1e-3, evt.lep_eta[1], evt.lep_phi[1])
    # invmass_tlorentz=getInvMassTL(evt.lep_pt[0]*1e-3, evt.lep_eta[0], evt.lep_phi[0], evt.lep_E[0]*1e-3, evt.lep_pt[1]*1e-3, evt.lep_eta[1], evt.lep_phi[1], evt.lep_E[1]*1e-3)
    # print("invmass1 = {}, invmass2 = {}".format(invmass_func, invmass_tlorentz))
    h_mass_electrons.Fill(getInvMass(evt.lep_pt[0]*1e-3, evt.lep_eta[0], evt.lep_phi[0], evt.lep_pt[1]*1e-3, evt.lep_eta[1], evt.lep_phi[1]))
    
    ievt += 1
    

In [10]:
# Gör samma sak för MC, så kan vi rita upp den med
h_mass_electrons_simulation = ROOT.TH1F("h_mass_electrons_simulation", "; Invariant mass [GeV]; Number of events", nbins, lowedge, upedge)
h_mass_muons_simulation = ROOT.TH1F("h_mass_muons_simulation", "; Invariant mass [GeV]; Number of events", nbins, lowedge, upedge)
ievt = 0
for evt in tree_electrons_simulation:
    if evt.lep_n != 2: continue
    if not (evt.lep_type[0]==11 and evt.lep_type[1]==11): continue
    if (evt.lep_charge[0] + evt.lep_charge[1]) != 0: continue 
    if ievt > 6000:
        break
        
    # invmass_func =     getInvMass(evt.lep_pt[0]*1e-3, evt.lep_eta[0], evt.lep_phi[0],                    evt.lep_pt[1]*1e-3, evt.lep_eta[1], evt.lep_phi[1])
    # invmass_tlorentz=getInvMassTL(evt.lep_pt[0]*1e-3, evt.lep_eta[0], evt.lep_phi[0], evt.lep_E[0]*1e-3, evt.lep_pt[1]*1e-3, evt.lep_eta[1], evt.lep_phi[1], evt.lep_E[1]*1e-3)
    # print("invmass1 = {}, invmass2 = {}".format(invmass_func, invmass_tlorentz))
    h_mass_electrons_simulation.Fill(getInvMass(evt.lep_pt[0]*1e-3, evt.lep_eta[0], evt.lep_phi[0], evt.lep_pt[1]*1e-3, evt.lep_eta[1], evt.lep_phi[1]))
    
    ievt += 1

In [11]:
%jsroot on

In [12]:
canvas = ROOT.TCanvas("canvas", "", 800, 600)
h_mass_electrons.Draw("e1p")
h_mass_electrons_simulation.Scale(0.5)
h_mass_electrons_simulation.SetFillColor(ROOT.kRed)
h_mass_electrons_simulation.SetMarkerColor(ROOT.kRed)
h_mass_electrons_simulation.SetLineColor(ROOT.kRed)
h_mass_electrons_simulation.Draw("he1 SAME")
canvas.Draw()

In [13]:
# Gör en anpassning till peaken med rätt funktion
# prova först dummy Gauss
h_mass_electrons.Fit("gaus", "S", "SAME", 80., 100.)

 FCN=86.2196 FROM MIGRAD    STATUS=CONVERGED      61 CALLS          62 TOTAL
                     EDM=6.18525e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     7.47756e+02   1.96828e+01   6.46060e-02  -2.02831e-05
   2  Mean         8.97312e+01   7.23979e-02   3.19402e-04  -7.79677e-04
   3  Sigma        3.59076e+00   6.95907e-02   2.08346e-05  -7.53853e-02


In [14]:
canvas.Draw()

In [33]:
# Set Breit-Wigner limits
norm_low = 100; norm_up = 15e3
mean_low = 80; mean_up = 90
width_low = 0.1; width_up = 40
def setBWLims(func):
    ''' Set limits on BW parameters '''
    func.SetParLimits(0, norm_low, norm_up)
    func.SetParLimits(1, mean_low, mean_up)
    func.SetParLimits(2, width_low, width_up)

In [50]:
#foo=ROOT.TMath.BreitWigner(4., 3., 3.)
fBW = ROOT.TF1("fBW", "[0]*TMath::BreitWigner(x, [1], [2])", 50., 120.)
fBW.SetParameters(5000, 90, 5)
fBW.SetParLimits(0, 100, 10000)
fBW.SetParLimits(1, 80, 100)
fBW.SetParLimits(2, 0.1, 40)
# first order polynomial as background
fFit1 = ROOT.TF1("fit1", "fBW(0)+pol1(3)", 50., 120.)
setBWLims(fFit1)
fFit2 = ROOT.TF1("fit2", "fBW(0)+pol2(3)", 50., 120.)
setBWLims(fFit2)
fFull = ROOT.TF1("fullManual", "[0]*[2]/((x-[1])*(x-[1]) + [2]*[2]/4) + [3]*x + [4]", 50., 120.)
setBWLims(fFull)

The history saving thread hit an unexpected error (OperationalError('unable to open database file')).History will not be written to the database.

Exception in thread IPythonHistorySavingThread:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/IPython/core/history.py", line 834, in run
  File "<decorator-gen-23>", line 2, in writeout_cache
  File "/usr/local/lib/python3.7/site-packages/IPython/core/history.py", line 58, in needs_sqlite
  File "/usr/local/lib/python3.7/site-packages/IPython/core/history.py", line 780, in writeout_cache
  File "/usr/local/lib/python3.7/site-packages/IPython/core/history.py", line 764, in _writeout_input_cache
sqlite3.OperationalError: unable to open database file

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/ipykernel/iostream.py", line 97, in _event_pipe
AttributeError: '_thread._local' object has no attribute 'event_pipe'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.

In [49]:
h_mass_electrons.Fit("fullManual", "S+", "SAME", 50., 120.)

 FCN=2404.01 FROM MIGRAD    STATUS=CALL LIMIT   1627 CALLS        1628 TOTAL
                     EDM=0.80512    STRATEGY= 1  ERROR MATRIX UNCERTAINTY  21.6 per cent
  EXT PARAMETER                APPROXIMATE        STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0          -1.47145e+06   1.45596e+00  -1.59779e-01   3.29157e-04
   2  p1          -1.18152e+02   4.86196e+00  -2.15189e+00  -5.10493e+00
   3  p2           1.55088e+01   1.26924e+00   5.81454e-01  -3.11277e+01
   4  p3          -5.76564e+00   2.58484e-01  -3.46596e-02   5.71235e+01
   5  p4           1.09228e+03   4.92940e+01   1.21334e+01   7.56676e-01


Warning in <Fit>: Abnormal termination of minimization.


In [45]:
#fFit1.Draw("SAME")
canvas.Draw()